In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Params
vocab = list(set([token for token in data['tokens'] for token in token]))
vocab_size = len(vocab)
embedding_dim = 300


# Metrics
metrics = [
      RecallMulticlass(name='recall', n_class=train_dataset.element_spec[1].shape[0]),
      PrecisionMulticlass(name='precision', n_class=train_dataset.element_spec[1].shape[0])
      ]

# Embedding layer
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False,
    name='embedding'
)

# Model
baseline_model = Sequential([
    embedding_layer,
    LSTM(units=64, name="lstm1"),
    Dense(units=64, activation='relu', name='dense1'),
    Dense(units=3, activation='softmax', name='output')
], name='baseline_model')


baseline_model.compile(optimizer='adam', loss=WeightedCategoricalCrossEntropy(weights=weights),
                       metrics=metrics)

baseline_model.summary()

# train the model
baseline_model_history = baseline_model.fit(batched_dataset, epochs=10, callbacks=[early_stopping, tensorboard_callback],
                                            validation_data=batched_evaluation)

# Evaluate the model
#baseline_model.evaluate(batched_evaluation)

In [ ]:
# Evaluate the model
eval_results = baseline_model.evaluate(batched_evaluation)
eval_loss, eval_precision, eval_recall = eval_results  # Assuming three metrics
eval_precision, eval_recall

In [ ]:
baseline_model.save(os.path.join(project_root, 'models', 'baseline_model'))

In [ ]:
import mlflow
import mlflow.tensorflow

project_root = os.path.dirname(os.getcwd())
mlruns_path = os.path.join(project_root, 'mlruns')

# Set the tracking URI to the custom path
mlflow.set_tracking_uri(f"file://{mlruns_path}")

## Create or get an existing experiment by name
experiment_name = "baseline_experiment"
#experiment_id = mlflow.create_experiment(experiment_name, artifact_location=f"file://{mlruns_path}")

# Set the experiment
mlflow.set_experiment(experiment_name)


# Start an MLflow run
with mlflow.start_run(run_name="baseline_model") as run:
    # Log model
    mlflow.tensorflow.log_model(baseline_model, "model")
    # Log metrics
    
    for epoch, metrics in enumerate(baseline_model_history.history["loss"]):
        mlflow.log_metric("loss", metrics, step=epoch)
        
    for epoch, metrics in enumerate(baseline_model_history.history["recall"]):
        mlflow.log_metric("recall", metrics, step=epoch)
        
    for epoch, metrics in enumerate(baseline_model_history.history["precision"]):
        mlflow.log_metric("precision", metrics, step=epoch)
    

## Evaluation plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def plot_metrics_with_seaborn(history):

    # Extract metrics from history
    metrics = [key for key in history.history.keys() if 'val_' not in key]
    epochs = range(1, len(history.history[metrics[0]]) + 1)
    

    for metric in metrics:
        plt.figure(figsize=(6, 3))
        
        # Plot training metric
        sns.lineplot(x=epochs, y=history.history[metric], label=f'Training {metric.capitalize()}', marker='o')
        
        # Plot validation metric if available
        if f'val_{metric}' in history.history:
            sns.lineplot(x=epochs, y=history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}', marker='o')
        
        # Formatting
        plt.title(f'{metric.capitalize()} Over Epochs', fontsize=16)
        plt.xlabel('Epochs', fontsize=14)
        plt.ylabel(metric.capitalize(), fontsize=14)
        plt.legend(loc='best', fontsize=12)
        plt.grid(True)
        plt.xticks(epochs)  # Show each epoch number on x-axis
        plt.tight_layout()  # Adjust layout for better fit
        plt.show()

plot_metrics_with_seaborn(baseline_model_history)